In [1]:
import numpy as np
from pathlib import Path
import mne
from util import *
from scipy.signal import butter, sosfiltfilt, sosfreqz  # for filtering
import matplotlib.pyplot as plt   
import pandas as pd
from sklearn.metrics import cohen_kappa_score

### Data Processing

In [2]:
passbands = [
    (1, 3), (2, 5), (4, 7), (6, 10), (7, 12),
    (10, 15), (12, 19), (15, 25), (19, 30),
    (25, 35), (30, 40)
]

gdf_folder = Path("BCICIV_2a_gdf")

target_events = {'769': 'left', '770': 'right', '771': 'feet', '772': 'tongue'}

filtered_data = []
labels_2d = []

# skipping AO4T because only has eye movement
users = ["A01T.gdf", "A02T.gdf", "A03T.gdf", "A05T.gdf", "A06T.gdf", "A07T.gdf", "A08T.gdf", "A09T.gdf"]

for subject in users:
    # Iterate through users
    gdf_path = gdf_folder / subject
    print(f"\nProcessing {subject}...")
    
    raw = mne.io.read_raw_gdf(gdf_path.resolve(), preload=True)
    events, event_id_map = mne.events_from_annotations(raw)

    subject_data = []
    user_labels = []

    user_event_ids = {target_events[key]: event_id_map[key] for key in target_events if key in event_id_map}

    # Filter and epoch for each band
    for band in passbands:
        low, high = band
        print(f"  Filtering {low}-{high} Hz...")

        # Bandpass filter
        raw_filtered = raw.copy().filter(l_freq=low, h_freq=high, method='fir', fir_design='firwin')

        # Create epochs
        epochs = mne.Epochs(raw_filtered, events, event_id=user_event_ids, baseline=(None, 0), event_repeated='merge', preload=True, tmin=-.3, tmax=4)
        labels = epochs.events[:, -1]  # Last column contains the event ID

        # Collect data and labels
        subject_data.append(epochs.get_data()[:, :-3, :])  # Shape: (n_trials, n_channels, n_times)
        user_labels.append(labels)

    filtered_data.append(subject_data)
    labels_2d.append(user_labels)


Processing A01T.gdf...
Extracting EDF parameters from C:\Users\Owner\COGS189-BCI-Competition-Project\BCICIV_2a_gdf\A01T.gdf...
GDF file detected
Setting channel info structure...
Could not determine channel type of the following channels, they will be set as EEG:
EEG-Fz, EEG, EEG, EEG, EEG, EEG, EEG, EEG-C3, EEG, EEG-Cz, EEG, EEG-C4, EEG, EEG, EEG, EEG, EEG, EEG, EEG, EEG-Pz, EEG, EEG, EOG-left, EOG-central, EOG-right
Creating raw.info structure...
Reading 0 ... 672527  =      0.000 ...  2690.108 secs...


c:\Users\Owner\AppData\Local\Programs\Python\Python311\Lib\contextlib.py:144: RuntimeWarning: Channel names are not unique, found duplicates for: {'EEG'}. Applying running numbers for duplicates.
  next(self.gen)


Used Annotations descriptions: ['1023', '1072', '276', '277', '32766', '768', '769', '770', '771', '772']
  Filtering 1-3 Hz...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 3 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 3.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 4.00 Hz)
- Filter length: 825 samples (3.300 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.7s


Not setting metadata
288 matching events found
Setting baseline interval to [-0.3, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 288 events and 1076 original time points ...
0 bad epochs dropped
  Filtering 2-5 Hz...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 2 - 5 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 2.00
- Lower transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 1.00 Hz)
- Upper passband edge: 5.00 Hz
- Upper transition bandwidth: 2.00 Hz (-6 dB cutoff frequency: 6.00 Hz)
- Filter length: 413 samples (1.652 s)



...

Not setting metadata
288 matching events found
Setting baseline interval to [-0.3, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 288 events and 1076 original time points ...
0 bad epochs dropped
  Filtering 25-35 Hz...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 25 - 35 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 25.00
- Lower transition bandwidth: 6.25 Hz (-6 dB cutoff frequency: 21.88 Hz)
- Upper passband edge: 35.00 Hz
- Upper transition bandwidth: 8.75 Hz (-6 dB cutoff frequency: 39.38 Hz)
- Filter length: 133 samples (0.532 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    1.2s


Not setting metadata
288 matching events found
Setting baseline interval to [-0.3, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 288 events and 1076 original time points ...
0 bad epochs dropped
  Filtering 30-40 Hz...
Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 30 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 30.00
- Lower transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 26.25 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 111 samples (0.444 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    1.3s


Not setting metadata
288 matching events found
Setting baseline interval to [-0.3, 0.0] s
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 288 events and 1076 original time points ...
0 bad epochs dropped


In [3]:
filtered_data = np.array(filtered_data)
filtered_data.shape

(8, 11, 288, 22, 1076)

In [4]:
labels_2d = np.array(labels_2d)
labels_2d.shape

(8, 11, 288)

**Separating data into 4 groups**

In [5]:
data_left = []
data_right = []
data_feet = []
data_tongue = []

for subject in range(len(filtered_data)):
    subject_data_left = []
    subject_data_right = []
    subject_data_feet = []
    subject_data_tongue = []
    for band in range(len(filtered_data[subject])):
        band_data_left = []
        band_data_right = []
        band_data_feet = []
        band_data_tongue = []
        for label in range(len(labels_2d[subject][band])):
            if labels_2d[subject][band][label] == 7:
                band_data_left.append(filtered_data[subject][band][label])
            elif labels_2d[subject][band][label] == 8:
                band_data_right.append(filtered_data[subject][band][label])
            elif labels_2d[subject][band][label] == 9:
                band_data_feet.append(filtered_data[subject][band][label])
            else:
                band_data_tongue.append(filtered_data[subject][band][label])
        subject_data_left.append(band_data_left)
        subject_data_right.append(band_data_right)
        subject_data_feet.append(band_data_feet)
        subject_data_tongue.append(band_data_tongue)
    data_left.append(subject_data_left)
    data_right.append(subject_data_right)
    data_feet.append(subject_data_feet)
    data_tongue.append(subject_data_tongue)

data_left = np.array(data_left)
data_right = np.array(data_right)
data_feet = np.array(data_feet)
data_tongue = np.array(data_tongue)

data_left.shape, data_right.shape, data_feet.shape, data_tongue.shape

((8, 11, 72, 22, 1076),
 (8, 11, 72, 22, 1076),
 (8, 11, 72, 22, 1076),
 (8, 11, 72, 22, 1076))

In [6]:
data_OVR_left = []
data_OVR_right = []
data_OVR_feet = []
data_OVR_tongue = []

random_samples = np.random.permutation(72)
random_sample_range1 = random_samples[:24]
random_sample_range2 = random_samples[24:48]
random_sample_range3 = random_samples[48:72]

# data_OVR_left
for subject in range(8):
    subject_data = []
    for band in range(11):
        band_data = []
        for sample in random_sample_range1:
            band_data.append(data_right[subject][band][sample])
        for sample in random_sample_range2:
            band_data.append(data_feet[subject][band][sample])
        for sample in random_sample_range3:
            band_data.append(data_tongue[subject][band][sample])
        np.random.shuffle(band_data)
        subject_data.append(band_data)
    data_OVR_left.append(subject_data)

# data_OVR_right
for subject in range(8):
    subject_data = []
    for band in range(11):
        band_data = []
        for sample in random_sample_range1:
            band_data.append(data_left[subject][band][sample])
        for sample in random_sample_range2:
            band_data.append(data_feet[subject][band][sample])
        for sample in random_sample_range3:
            band_data.append(data_tongue[subject][band][sample])
        np.random.shuffle(band_data)
        subject_data.append(band_data)
    data_OVR_right.append(subject_data)

# data_OVR_feet
for subject in range(8):
    subject_data = []
    for band in range(11):
        band_data = []
        for sample in random_sample_range1:
            band_data.append(data_left[subject][band][sample])
        for sample in random_sample_range2:
            band_data.append(data_right[subject][band][sample])
        for sample in random_sample_range3:
            band_data.append(data_tongue[subject][band][sample])
        np.random.shuffle(band_data)
        subject_data.append(band_data)
    data_OVR_feet.append(subject_data)

# data_OVR_tongue
for subject in range(8):
    subject_data = []
    for band in range(11):
        band_data = []
        for sample in random_sample_range1:
            band_data.append(data_left[subject][band][sample])
        for sample in random_sample_range2:
            band_data.append(data_right[subject][band][sample])
        for sample in random_sample_range3:
            band_data.append(data_feet[subject][band][sample])
        np.random.shuffle(band_data)
        subject_data.append(band_data)
    data_OVR_tongue.append(subject_data)

# Convert to numpy arrays
data_OVR_left = np.array(data_OVR_left)
data_OVR_right = np.array(data_OVR_right)
data_OVR_feet = np.array(data_OVR_feet)
data_OVR_tongue = np.array(data_OVR_tongue)

In [7]:
# # Reordering dimensions
data_left = data_left.transpose(0, 1, 3, 4, 2)
data_right = data_right.transpose(0, 1, 3, 4, 2)
data_feet = data_feet.transpose(0, 1, 3, 4, 2)
data_tongue = data_tongue.transpose(0, 1, 3, 4, 2)

data_left.shape, data_right.shape, data_feet.shape, data_tongue.shape

((8, 11, 22, 1076, 72),
 (8, 11, 22, 1076, 72),
 (8, 11, 22, 1076, 72),
 (8, 11, 22, 1076, 72))

In [8]:
# # Reordering dimensions
data_OVR_left = data_OVR_left.transpose(0, 1, 3, 4, 2)
data_OVR_right = data_OVR_right.transpose(0, 1, 3, 4, 2)
data_OVR_feet = data_OVR_feet.transpose(0, 1, 3, 4, 2)
data_OVR_tongue = data_OVR_tongue.transpose(0, 1, 3, 4, 2)

data_OVR_left.shape, data_OVR_right.shape, data_OVR_feet.shape, data_OVR_tongue.shape

((8, 11, 22, 1076, 72),
 (8, 11, 22, 1076, 72),
 (8, 11, 22, 1076, 72),
 (8, 11, 22, 1076, 72))

### Start of Assignment 3 Code

In [9]:
n_sample = 72

In [10]:
#n_channel, n_time, n_sample = EEGR_train[0][0].shape
randIdx = np.random.permutation(n_sample)
randIdx

array([71, 29, 43, 28, 44, 17,  6, 35, 46, 10, 13, 30, 33, 12,  3, 64, 36,
       14, 34, 24, 55, 27, 59, 53, 56, 15, 45, 21, 51,  0, 61, 41, 48, 68,
       11, 42, 66, 67, 69,  9, 58, 38, 54, 62, 47, 20, 63, 60,  8, 23, 70,
       26,  1, 19, 49,  2, 50, 32, 57, 31, 22, 16, 65, 18, 52,  7, 37, 25,
       40,  4,  5, 39])

In [12]:
k = 6
print(f'Our k is: {k}')

bin_size = int(n_sample / k)
print(f'Our bin size is: {bin_size}')

# In order to make our task easier, 
# we can reshape randIdx to have k rows and bin_size columns
# by using .reshape():
randIdx = randIdx.reshape(k, bin_size)
print(randIdx)

Our k is: 6
Our bin size is: 12
[[71 29 43 28 44 17  6 35 46 10 13 30]
 [33 12  3 64 36 14 34 24 55 27 59 53]
 [56 15 45 21 51  0 61 41 48 68 11 42]
 [66 67 69  9 58 38 54 62 47 20 63 60]
 [ 8 23 70 26  1 19 49  2 50 32 57 31]
 [22 16 65 18 52  7 37 25 40  4  5 39]]


In [13]:
trainIdx = np.zeros((k, bin_size*(k-1)), dtype=int)
valIdx = np.zeros((k, bin_size), dtype=int)

In [32]:
# We'll reset trainIdx and valIdx (just in case)
# Note: feel free to use uninitialized lists and append
#       this is just one possible starting point to show
#       the correct size of these objects.
trainIdx = np.zeros((k, bin_size*(k-1)), dtype=int) # could also be = []
valIdx = np.zeros((k, bin_size), dtype=int)           # could also be = []

# --- Question 4 ---
# YOUR CODE HERE

for i in range(k):
    lst = []
    for row in range(k):
        if row == 0:
            valIdx[i] = randIdx[row]
            continue
        for col in range(len(randIdx[0])):
            lst.append(randIdx[row][col])
    trainIdx[i] = lst
    randIdx = np.random.permutation(n_sample)
    randIdx = randIdx.reshape(k, bin_size)
valIdx

array([[26, 30,  1, 61, 55, 31, 63, 15, 54, 58, 70, 68],
       [43,  2, 27, 49, 50, 33, 32, 41, 55, 38, 10, 71],
       [59, 38, 31, 35, 42, 54,  8, 34,  7, 12, 28, 70],
       [ 0, 65, 68, 12, 71,  2, 43, 36, 19, 10, 17, 33],
       [ 8, 57, 21, 49, 43, 31, 56, 16, 27, 44,  6,  4],
       [ 5, 43, 10, 62, 58, 30, 39, 61, 69, 42,  7, 22]])

In [33]:
# Depending on your solution, valIdx and trainIdx may not be np.arrays
# Let's guarantee they are
valIdx = np.array(valIdx)
trainIdx = np.array(trainIdx)

assert valIdx.shape == (k, bin_size)
assert trainIdx.shape == (k, bin_size*(k-1))

In [34]:
trainIdx.shape

(6, 60)

In [35]:
valIdx.shape

(6, 12)

In [36]:
data_left[0, 0].shape

(22, 1076, 72)

### This is left vs right classification from assignment 3

In [37]:
# Main Analysis (Do Not Modify)

# Declare useful variables
n_subjects, n_filters = (8, 11)
train_size = bin_size * (k-1)
val_size = bin_size
accuracy = np.zeros((n_subjects, k))
csp_per_class = 3
trainIdx = trainIdx.astype(int)
valIdx = valIdx.astype(int)

# Begin nested loops
# Subject > Fold > Band
print('===============')
for subject in range(n_subjects):
    print('Processing Subject {}/{}'.format(subject+1, n_subjects))
    for fold in range(k):
        print('Processing Fold {}/{}'.format(fold+1, k))
        band_score_train = np.zeros((train_size*2, n_filters))
        band_score_val = np.zeros((val_size*2, n_filters))
        for band in range(n_filters):
            # train,val dataset -> train,val for each subject&band
            L_train = data_left[subject, band][:,:,trainIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            R_train = data_right[subject, band][:,:,trainIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            L_val = data_left[subject, band][:,:,valIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            R_val = data_right[subject, band][:,:,valIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            train_data = np.stack((L_train,R_train), axis=0) # <- CSP input data format (classes, trials, chans, samples) 
            val_data = np.stack((L_val,R_val), axis=0) # <- CSP input data format (classes, trials, chans, samples)

            # Run Mahta's CSP code
            # Source: https://github.com/mahtamsv/CCACSP
            csp_filts, clf = train(train_data[0], train_data[1], csp_per_class)
            train_prob = test(np.vstack(train_data), csp_filts, clf)
            val_prob = test(np.vstack(val_data), csp_filts, clf)
                        
            # test() gives the prob of the 0th class, that's why 
            # we have y_val seemingly switched below
            y_val = np.append(np.ones(12), np.zeros(12))

            band_score_train[:,band] = train_prob
            band_score_val[:,band] = val_prob
        
        
        accuracy[subject,fold] = sum(np.rint(band_score_val.mean(1))==y_val)/len(y_val)
    print('===============')
print(np.mean(accuracy, axis=1))

# Main Analysis Over

Processing Subject 1/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 2/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 3/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 4/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 5/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 6/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 7/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6

### Need to create 4 more: Left vs rest, Right vs rest, Feet vs rest, Tongue vs rest

This will involve creating 4 more train sets. 

E.g. Instead of L_train and R_train, we can use L_train again, but we need a O_train which contains an equal amount of data points from the other 3 classes.

## Left vs Rest

In [ ]:
# We'll reset trainIdx and valIdx (just in case)
# Note: feel free to use uninitialized lists and append
#       this is just one possible starting point to show
#       the correct size of these objects.
trainIdx = np.zeros((k, bin_size*(k-1)), dtype=int) # could also be = []
valIdx = np.zeros((k, bin_size), dtype=int)           # could also be = []

# --- Question 4 ---
# YOUR CODE HERE

for i in range(k):
    lst = []
    for row in range(k):
        if row == 0:
            valIdx[i] = randIdx[row]
            continue
        for col in range(len(randIdx[0])):
            lst.append(randIdx[row][col])
    trainIdx[i] = lst
    randIdx = np.random.permutation(n_sample)
    randIdx = randIdx.reshape(k, bin_size)

assert valIdx.shape == (k, bin_size)
assert trainIdx.shape == (k, bin_size*(k-1))

# Main Analysis (Do Not Modify)

# Declare useful variables
n_subjects, n_filters = (8, 11)
train_size = bin_size * (k-1)
val_size = bin_size
accuracy = np.zeros((n_subjects, k))
kappa = np.zeros((n_subjects, k))
csp_per_class = 3
trainIdx = trainIdx.astype(int)
valIdx = valIdx.astype(int)

# Begin nested loops
# Subject > Fold > Band
# n_subjects = 1
print('===============')
for subject in range(n_subjects):
    print('Processing Subject {}/{}'.format(subject+1, n_subjects))
    for fold in range(k):
        print('Processing Fold {}/{}'.format(fold+1, k))
        band_score_train = np.zeros((train_size*2, n_filters))
        band_score_val = np.zeros((val_size*2, n_filters))
        for band in range(n_filters):
            # train,val dataset -> train,val for each subject&band
            L_train = data_left[subject, band][:,:,trainIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            OVR_train = data_OVR_left[subject, band][:,:,trainIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            L_val = data_left[subject, band][:,:,valIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            OVR_val = data_OVR_left[subject, band][:,:,valIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            train_data = np.stack((L_train,OVR_train), axis=0) # <- CSP input data format (classes, trials, chans, samples) 
            val_data = np.stack((L_val,OVR_val), axis=0) # <- CSP input data format (classes, trials, chans, samples)

            # Run Mahta's CSP code
            # Source: https://github.com/mahtamsv/CCACSP
            csp_filts, clf = train(train_data[0], train_data[1], csp_per_class)
            train_prob = test(np.vstack(train_data), csp_filts, clf)
            val_prob = test(np.vstack(val_data), csp_filts, clf)
            
            # test() gives the prob of the 0th class, that's why 
            # we have y_val seemingly switched below
            y_val = np.append(np.ones(12), np.zeros(12))

            band_score_train[:,band] = train_prob
            band_score_val[:,band] = val_prob

        y_pred = np.rint(band_score_val.mean(1))
        accuracy[subject,fold] = sum(y_pred==y_val)/len(y_val)
        kappa[subject, fold] = cohen_kappa_score(y_pred, y_val)
    print('===============')
print(np.mean(accuracy, axis=1))
print(np.mean(kappa, axis=1))

# Main Analysis Over

Processing Subject 1/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 2/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 3/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 4/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 5/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 6/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 7/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6

## Right vs Rest

In [24]:
# We'll reset trainIdx and valIdx (just in case)
# Note: feel free to use uninitialized lists and append
#       this is just one possible starting point to show
#       the correct size of these objects.
trainIdx = np.zeros((k, bin_size*(k-1)), dtype=int) # could also be = []
valIdx = np.zeros((k, bin_size), dtype=int)           # could also be = []

# --- Question 4 ---
# YOUR CODE HERE

for i in range(k):
    lst = []
    for row in range(k):
        if row == 0:
            valIdx[i] = randIdx[row]
            continue
        for col in range(len(randIdx[0])):
            lst.append(randIdx[row][col])
    trainIdx[i] = lst
    randIdx = np.random.permutation(n_sample)
    randIdx = randIdx.reshape(k, bin_size)
valIdx

assert valIdx.shape == (k, bin_size)
assert trainIdx.shape == (k, bin_size*(k-1))

# Main Analysis (Do Not Modify)

# Declare useful variables
n_subjects, n_filters = (8, 11)
train_size = bin_size * (k-1)
val_size = bin_size
accuracy = np.zeros((n_subjects, k))
kappa = np.zeros((n_subjects, k))
csp_per_class = 3
trainIdx = trainIdx.astype(int)
valIdx = valIdx.astype(int)

# Begin nested loops
# Subject > Fold > Band
print('===============')
for subject in range(n_subjects):
    print('Processing Subject {}/{}'.format(subject+1, n_subjects))
    for fold in range(k):
        print('Processing Fold {}/{}'.format(fold+1, k))
        band_score_train = np.zeros((train_size*2, n_filters))
        band_score_val = np.zeros((val_size*2, n_filters))
        for band in range(n_filters):
            # train,val dataset -> train,val for each subject&band
            r_train = data_right[subject, band][:,:,trainIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            OVR_train = data_OVR_right[subject, band][:,:,trainIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            r_val = data_right[subject, band][:,:,valIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            OVR_val = data_OVR_right[subject, band][:,:,valIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            train_data = np.stack((r_train,OVR_train), axis=0) # <- CSP input data format (classes, trials, chans, samples) 
            val_data = np.stack((r_val,OVR_val), axis=0) # <- CSP input data format (classes, trials, chans, samples)

            # Run Mahta's CSP code
            # Source: https://github.com/mahtamsv/CCACSP
            csp_filts, clf = train(train_data[0], train_data[1], csp_per_class)
            train_prob = test(np.vstack(train_data), csp_filts, clf)
            val_prob = test(np.vstack(val_data), csp_filts, clf)
            
            # test() gives the prob of the 0th class, that's why 
            # we have y_val seemingly switched below
            y_val = np.append(np.ones(12), np.zeros(12))

            band_score_train[:,band] = train_prob
            band_score_val[:,band] = val_prob
        
        
        y_pred = np.rint(band_score_val.mean(1))
        accuracy[subject,fold] = sum(y_pred==y_val)/len(y_val)
        kappa[subject, fold] = cohen_kappa_score(y_pred, y_val)
    print('===============')
print(np.mean(accuracy, axis=1))
print(np.mean(kappa, axis=1))

# Main Analysis Over

Processing Subject 1/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 2/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 3/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 4/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 5/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 6/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 7/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6

## Feet vs Rest

In [25]:
# We'll reset trainIdx and valIdx (just in case)
# Note: feel free to use uninitialized lists and append
#       this is just one possible starting point to show
#       the correct size of these objects.
trainIdx = np.zeros((k, bin_size*(k-1)), dtype=int) # could also be = []
valIdx = np.zeros((k, bin_size), dtype=int)           # could also be = []

# --- Question 4 ---
# YOUR CODE HERE

for i in range(k):
    lst = []
    for row in range(k):
        if row == 0:
            valIdx[i] = randIdx[row]
            continue
        for col in range(len(randIdx[0])):
            lst.append(randIdx[row][col])
    trainIdx[i] = lst
    randIdx = np.random.permutation(n_sample)
    randIdx = randIdx.reshape(k, bin_size)
valIdx

assert valIdx.shape == (k, bin_size)
assert trainIdx.shape == (k, bin_size*(k-1))

# Main Analysis (Do Not Modify)

# Declare useful variables
n_subjects, n_filters = (8, 11)
train_size = bin_size * (k-1)
val_size = bin_size
accuracy = np.zeros((n_subjects, k))
kappa = np.zeros((n_subjects, k))
csp_per_class = 3
trainIdx = trainIdx.astype(int)
valIdx = valIdx.astype(int)

# Begin nested loops
# Subject > Fold > Band
print('===============')
for subject in range(n_subjects):
    print('Processing Subject {}/{}'.format(subject+1, n_subjects))
    for fold in range(k):
        print('Processing Fold {}/{}'.format(fold+1, k))
        band_score_train = np.zeros((train_size*2, n_filters))
        band_score_val = np.zeros((val_size*2, n_filters))
        for band in range(n_filters):
            # train,val dataset -> train,val for each subject&band
            f_train = data_feet[subject, band][:,:,trainIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            OVR_train = data_OVR_feet[subject, band][:,:,trainIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            f_val = data_feet[subject, band][:,:,valIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            OVR_val = data_OVR_feet[subject, band][:,:,valIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            train_data = np.stack((f_train,OVR_train), axis=0) # <- CSP input data format (classes, trials, chans, samples) 
            val_data = np.stack((f_val,OVR_val), axis=0) # <- CSP input data format (classes, trials, chans, samples)

            # Run Mahta's CSP code
            # Source: https://github.com/mahtamsv/CCACSP
            csp_filts, clf = train(train_data[0], train_data[1], csp_per_class)
            train_prob = test(np.vstack(train_data), csp_filts, clf)
            val_prob = test(np.vstack(val_data), csp_filts, clf)
            
            # test() gives the prob of the 0th class, that's why 
            # we have y_val seemingly switched below
            y_val = np.append(np.ones(12), np.zeros(12))

            band_score_train[:,band] = train_prob
            band_score_val[:,band] = val_prob
        
        
        y_pred = np.rint(band_score_val.mean(1))
        accuracy[subject,fold] = sum(y_pred==y_val)/len(y_val)
        kappa[subject, fold] = cohen_kappa_score(y_pred, y_val)
    print('===============')
print(np.mean(accuracy, axis=1))
print(np.mean(kappa, axis=1))

# Main Analysis Over

Processing Subject 1/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 2/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 3/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 4/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 5/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 6/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 7/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6

## Tongue vs Rest

In [26]:
# We'll reset trainIdx and valIdx (just in case)
# Note: feel free to use uninitialized lists and append
#       this is just one possible starting point to show
#       the correct size of these objects.
trainIdx = np.zeros((k, bin_size*(k-1)), dtype=int) # could also be = []
valIdx = np.zeros((k, bin_size), dtype=int)           # could also be = []

# --- Question 4 ---
# YOUR CODE HERE

for i in range(k):
    lst = []
    for row in range(k):
        if row == 0:
            valIdx[i] = randIdx[row]
            continue
        for col in range(len(randIdx[0])):
            lst.append(randIdx[row][col])
    trainIdx[i] = lst
    randIdx = np.random.permutation(n_sample)
    randIdx = randIdx.reshape(k, bin_size)
valIdx

assert valIdx.shape == (k, bin_size)
assert trainIdx.shape == (k, bin_size*(k-1))

# Main Analysis (Do Not Modify)

# Declare useful variables
n_subjects, n_filters = (8, 11)
train_size = bin_size * (k-1)
val_size = bin_size
accuracy = np.zeros((n_subjects, k))
kappa = np.zeros((n_subjects, k))
csp_per_class = 3
trainIdx = trainIdx.astype(int)
valIdx = valIdx.astype(int)

# Begin nested loops
# Subject > Fold > Band
print('===============')
for subject in range(n_subjects):
    print('Processing Subject {}/{}'.format(subject+1, n_subjects))
    for fold in range(k):
        print('Processing Fold {}/{}'.format(fold+1, k))
        band_score_train = np.zeros((train_size*2, n_filters))
        band_score_val = np.zeros((val_size*2, n_filters))
        for band in range(n_filters):
            # train,val dataset -> train,val for each subject&band
            t_train = data_tongue[subject, band][:,:,trainIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            OVR_train = data_OVR_tongue[subject, band][:,:,trainIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            t_val = data_tongue[subject, band][:,:,valIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            OVR_val = data_OVR_tongue[subject, band][:,:,valIdx[fold]].transpose(2,0,1) # (trials, chans, samples)
            train_data = np.stack((t_train,OVR_train), axis=0) # <- CSP input data format (classes, trials, chans, samples) 
            val_data = np.stack((t_val,OVR_val), axis=0) # <- CSP input data format (classes, trials, chans, samples)

            # Run Mahta's CSP code
            # Source: https://github.com/mahtamsv/CCACSP
            csp_filts, clf = train(train_data[0], train_data[1], csp_per_class)
            train_prob = test(np.vstack(train_data), csp_filts, clf)
            val_prob = test(np.vstack(val_data), csp_filts, clf)
            
            # test() gives the prob of the 0th class, that's why 
            # we have y_val seemingly switched below
            y_val = np.append(np.ones(12), np.zeros(12))

            band_score_train[:,band] = train_prob
            band_score_val[:,band] = val_prob
        
        
        y_pred = np.rint(band_score_val.mean(1))
        accuracy[subject,fold] = sum(y_pred==y_val)/len(y_val)
        kappa[subject, fold] = cohen_kappa_score(y_pred, y_val)
    print('===============')
print(np.mean(accuracy, axis=1))
print(np.mean(kappa, axis=1))

# Main Analysis Over

Processing Subject 1/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 2/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 3/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 4/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 5/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 6/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6
Processing Subject 7/8
Processing Fold 1/6
Processing Fold 2/6
Processing Fold 3/6
Processing Fold 4/6
Processing Fold 5/6
Processing Fold 6/6

In [45]:
import os

os.system('jupyter nbconvert --to html CSP_LDA.ipynb')

0